In [ ]:
# Height, Width
imageSize = (104,88)

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers

import zipfile
import os
from skimage import io
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%pip install -q -U keras-tuner

import keras_tuner as kt

In [ ]:
# Unzips file in path and extracts in to folder
with zipfile.ZipFile('/content/data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [ ]:
# Gets all the class names
datasetPath = '/content/data/data/' # Change for data sets


classes = os.listdir(datasetPath)
# Adding all data to dataFiles
dataFiles = np.empty(shape=(0,2))
for c in classes:
  # Creates the image file path in element [0] and the class of the image in element [1]
  imgFile = np.array(list(map(lambda x: (f'{datasetPath}{c}/{x}', c), os.listdir(f'{datasetPath}{c}'))))
  prevLength = len(dataFiles)
  
  dataFiles = np.concatenate((dataFiles, imgFile))
  
  print(f'{datasetPath}{c}: {len(dataFiles)} image files total. {len(dataFiles) - prevLength} images in class {c}')

print(f'{dataFiles[0]}, {dataFiles[717]}, {dataFiles[769]}, {dataFiles[3329]}')

dataSet = np.array(list(map(lambda x: (io.imread(x[0]), x[1]), dataFiles)))

/content/data/data/Non_Demented: 3200 image files total. 3200 images in class Non_Demented
/content/data/data/Very_Mild_Demented: 5440 image files total. 2240 images in class Very_Mild_Demented
/content/data/data/Moderate_Demented: 5504 image files total. 64 images in class Moderate_Demented
/content/data/data/Mild_Demented: 6400 image files total. 896 images in class Mild_Demented
['/content/data/data/Non_Demented/non_575.jpg' 'Non_Demented'], ['/content/data/data/Non_Demented/non_473.jpg' 'Non_Demented'], ['/content/data/data/Non_Demented/non_3044.jpg' 'Non_Demented'], ['/content/data/data/Very_Mild_Demented/verymild_1399.jpg'
 'Very_Mild_Demented']


<ipython-input-15-5d4385b20b0e>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataSet = np.array(list(map(lambda x: (io.imread(x[0]), x[1]), dataFiles)))


In [ ]:
## Making class dictionary to chnage class labels to numbers
strDict = {"Non_Demented": 0, "Very_Mild_Demented": 1,   "Mild_Demented": 2, "Moderate_Demented": 3} # Change for data sets

# dataSet is split into images and labels
images = dataSet[:,0]
labels = dataSet[:,1]


print(labels[0:5])
listLabels = list(labels)
labels = np.array(list(map(lambda x: strDict[x], listLabels)))
print(labels[0:5])

#CHANGE#

# Data is split into training and testing
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.33) # Take out random_state when testing without a fair test

# Training data is split into validation data
# images_train, images_val, labels_train, labels_val = train_test_split(images_train, labels_train, test_size=0.2, random_state=1)

print(labels_train[0:4])

['Non_Demented' 'Non_Demented' 'Non_Demented' 'Non_Demented'
 'Non_Demented']
[0 0 0 0 0]
[1 0 1 0]


In [ ]:
new_images_train = []
for im in images_train:
  im = resize(im, imageSize, anti_aliasing=True)
  new_images_train.append(im)
images_train = new_images_train

new_images_test = []
for im in images_test:
  im = resize(im, imageSize, anti_aliasing=True)
  new_images_test.append(im)
images_test = new_images_test

#CHANGE#
# new_images_val = []
# for im in images_val:
#   im = resize(im, imageSize, anti_aliasing=True)
#   new_images_val.append(im)
# images_val = new_images_val

# Fine Tuning Hyperparameters
Build model and add points where tuner can tune model for improvement <br>
Tunes: 3 filters, One dense layer, learning rate

In [ ]:
# Build model here
def model_builder(hp):
  # Deciding variabilites wanted in model
  # Number of blocks
  hp_blocks = hp.Int('blocks', min_value=1, max_value=4, step=1)

  # Filters
  hp_inp_filter = hp.Int('inp_filter', min_value=8, max_value=128, step=8)
  hp_one_filter = hp.Int('inp_filter', min_value=8, max_value=128, step=8)
  hp_block_filter = hp.Int('one_filter', min_value=8, max_value=256, step=8)
  
  # Kernel size?
  hp_inp_kernel = hp.Int('inp_kernel', min_value=1, max_value=4, step=1)
  hp_block_kernel = hp.Int('block_kernel', min_value=1, max_value=4, step=1)

  # Max Pooling
  hp_maxpooling = hp.Int('maxpooling', min_value=1, max_value=4, step=1)
  hp_maxpooling_one = hp.Int('maxpooling_one', min_value=1, max_value=4, step=1)

  # Dropout layers
  hp_dropout = hp.Float('dropout', 0, 0.9, step=0.1)

  # Number of nodes
  hp_inp_dense_1 = hp.Int('dense1', min_value=8, max_value=1028, step=8)
  hp_inp_dense_2 = hp.Int('dense2', min_value=8, max_value=1028, step=8)

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001, 0.0001])


  # Model
  def convBlock(model, filter, kernel, mpKernel): # Adds another convolutional layer
    model.add(layers.MaxPooling2D(mpKernel))
    model.add(layers.Conv2D(filter, kernel, activation='relu'))

  model = keras.Sequential()
  input_layer = layers.Conv2D(hp_inp_filter, (hp_inp_kernel,hp_inp_kernel), activation='relu', input_shape=(imageSize[0],imageSize[1],1))
  input_layer._name = 'input' # Setting name of layer
  model.add(input_layer)


  model.add(layers.MaxPooling2D(hp_maxpooling_one))
  model.add(layers.Conv2D(hp_one_filter, (1,1), activation='relu'))


  for i in range(hp_blocks):
    convBlock(model, hp_block_filter, (hp_block_kernel,hp_block_kernel), (hp_maxpooling,hp_maxpooling))

  model.add(layers.Flatten())
  model.add(layers.Dropout(hp_dropout))
  model.add(layers.Dense(hp_inp_dense_1, activation='relu'))
  # model.add(layers.Dropout(0.75))
  # model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(hp_inp_dense_2, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))


  # Compile
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(model_builder,
                    objective='val_accuracy',
                    max_trials=50,
                    executions_per_trial=1,
                    directory='my_dirx'
                     )


stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)



In [ ]:
images_train = np.array(images_train)
images_train = images_train.astype('float32')

images_test = np.array(images_test)
images_test = images_test.astype('float32') 

**Get best learning rate, number of nodes in dense layer and convolution filter**

In [ ]:
tuner.search(images_train, labels_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps)

Trial 24 Complete [00h 00m 00s]

Best val_accuracy So Far: 0.9731934666633606
Total elapsed time: 00h 23m 05s

Search: Running Trial #25

Value             |Best Value So Far |Hyperparameter
2                 |1                 |blocks
40                |72                |inp_filter
248               |176               |block_filter
3                 |3                 |inp_kernel
3                 |4                 |block_kernel
3                 |2                 |maxpooling
0                 |0.7               |dropout
80                |480               |dense1
928               |680               |dense2
0.001             |0.001             |learning_rate
216               |112               |one_filter
3                 |2                 |maxpooling_one



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 213, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 155, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build

RuntimeError: ignored

**Get best number of epochs**

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(images_train, labels_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
model.evaluate(images_test, labels_test)